## **INSTRUÇÕES**

---
### Todos os arquivos baixados devem ser upados na pasta 'Arquivos de entrada'
---


#### **Passo 1:** Acesse o SFTP do Trivago, vá para a página outfiles e baixe o arquivo **trivago_hotel_data_br.csv** e renomeie ele para:

* feed.csv

#### **Passo 2:** Acesse o SFTP do Trivago, vá para a página outfiles e baixe o arquivo **feed_cpa.csv**. Não é necessário renomeá-lo.


In [1]:
 from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
### importando bibliotecas que iremos utilizar
import pandas as pd
import datetime as dt
import numpy as np

In [3]:
### Feed e campanhas
feed = pd.DataFrame(pd.read_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/Arquivo de entrada/feed.csv', delimiter = ',', encoding="ISO 8859-1"))
campaigns = pd.read_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/Arquivo de entrada/feed_cpa.csv', delimiter = ',', encoding="ISO 8859-1")
### fins_seo
fins_seo = pd.DataFrame(pd.read_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/Arquivo de entrada/fins seo.csv', delimiter = ';', decimal=',', encoding="ISO 8859-1"))
fins_seo_com = pd.DataFrame(pd.read_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/Arquivo de entrada/fins seo_com.csv', delimiter = ';', decimal=',', encoding="ISO 8859-1"))


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
feed.head()

,partner_reference,name,address,address2,postcode,city,state,country,longitude,latitude,phone,web
0,OMN-1000,Plaza Inn American Loft,"Rua Palestra ItÃ¡lia, 263 (antiga Rua Turiassu...",Perdizes,5005001.0,Sao Paulo,SÃ£o Paulo,Brasil,-46.680259,-23.528191,1121251800,https://www.hurb.com/hotels/sao-paulo/plaza-in...
1,OMN-10003,Hotel CaiuÃ¡ Umuruama Express,RUA ARICANDUVA 4061,Umuarama,87502200.0,Umuarama,ParanÃ¡,Brasil,-53.306866,-23.759823,554436213900,https://www.hurb.com/hotels/umuarama/hotel-cai...
2,OMN-10004,Blue Open Hotel,"Rua Gaurama, 335",NaN,99700070.0,Erechim,Rio Grande do Sul,Brasil,-52.278942,-27.634254,555437123350,https://www.hurb.com/hotels/erechim/blue-open-...
3,OMN-10005,Hotel Premium Hplus Palmas,"Avenida LO (Leste Oeste) 03 Plano Diretor Sul, SN",Plano Diretor Norte,77020464.0,Palmas,Tocantins,Brasil,-48.325440,-10.192172,6134265000,https://www.hurb.com/hotels/palmas/hotel-premi...
4,OMN-10008,Star Itaperuna Hotel,"R. Cel. LuÃ­s Ferraz, 500",Itaperuna,28300000.0,ITAPERUNA,Rio de Janeiro,Brasil,-41.888607,-21.201809,552238242615,https://www.hurb.com/hotels/itaperuna/star-ita...


In [5]:
campaigns.head()

,locale,partner_reference,campaign
0,BR,JNP-JP984417,0
1,BR,OMN-142,0
2,BR,OMN-4789,0
3,BR,OMN-4811,0
4,BR,OMN-4814,0


In [6]:
# Renomeando colunas das fins_seo
fins_seo = fins_seo.rename(columns = {'SKU': 'partner_reference'})
fins_seo_com = fins_seo_com.rename(columns = {"SKU": 'partner_reference'})


In [7]:
# no_campaign -> hoteis que não estão em nenhuma campanha no momento
no_campaign = feed[~feed['partner_reference'].isin(campaigns['partner_reference'])]
no_campaign

,partner_reference,name,address,address2,postcode,city,state,country,longitude,latitude,phone,web


In [8]:
# separado hoteis a não serem bidados por POS
fins_seo_com_BR = fins_seo_com[['partner_reference', 'BR']]
fins_seo_com_US = fins_seo_com[['partner_reference', 'US']]

fins_seo_com_US = fins_seo_com_US[fins_seo_com_US['US'].notna()]
fins_seo_com_BR = fins_seo_com_BR[fins_seo_com_BR['BR'].notna()]



In [9]:
# new hotels -> hoteis que não aparecem em nenhuma campanha no momento, e também não estão em nenhum fins_seo
new_hotels_BR = no_campaign[(~no_campaign['partner_reference'].isin(fins_seo['partner_reference'])) & 
                            (~no_campaign['partner_reference'].isin(fins_seo_com_BR['partner_reference']))]

new_hotels_US = no_campaign[(~no_campaign['partner_reference'].isin(fins_seo['partner_reference'])) & 
                            (~no_campaign['partner_reference'].isin(fins_seo_com_US['partner_reference']))]

new_hotels_BR

,partner_reference,name,address,address2,postcode,city,state,country,longitude,latitude,phone,web


In [10]:
# Resumo BR
new_hotels_BR.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   partner_reference  0 non-null      object 
 1   name               0 non-null      object 
 2   address            0 non-null      object 
 3   address2           0 non-null      object 
 4   postcode           0 non-null      float64
 5   city               0 non-null      object 
 6   state              0 non-null      object 
 7   country            0 non-null      object 
 8   longitude          0 non-null      float64
 9   latitude           0 non-null      float64
 10  phone              0 non-null      object 
 11  web                0 non-null      object 
dtypes: float64(3), object(9)
memory usage: 0.0+ bytes


In [11]:
# Resumo US
new_hotels_US.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   partner_reference  0 non-null      object 
 1   name               0 non-null      object 
 2   address            0 non-null      object 
 3   address2           0 non-null      object 
 4   postcode           0 non-null      float64
 5   city               0 non-null      object 
 6   state              0 non-null      object 
 7   country            0 non-null      object 
 8   longitude          0 non-null      float64
 9   latitude           0 non-null      float64
 10  phone              0 non-null      object 
 11  web                0 non-null      object 
dtypes: float64(3), object(9)
memory usage: 0.0+ bytes


In [12]:
# Filtering for partner_reference inactive values (HU and HSYS)
new_hotels_BR = new_hotels_BR[(~new_hotels_BR['partner_reference'].str.startswith('HU')) & (~new_hotels_BR['partner_reference'].str.startswith('HSYS'))]
new_hotels_US = new_hotels_US[(~new_hotels_US['partner_reference'].str.startswith('HU')) & (~new_hotels_US['partner_reference'].str.startswith('HSYS'))]

In [13]:
num_omn = new_hotels_BR['partner_reference'].str.count('OMN').sum()
num_jnp = new_hotels_BR['partner_reference'].str.count('JNP').sum()

display(f'Number of OMN skus: {num_omn}')
display(f'Number of JNP skus: {num_jnp}')


'Number of OMN skus: 0'

'Number of JNP skus: 0'

In [14]:
### Preparing outputs
# BR
new_hotels_BR = new_hotels_BR[['partner_reference']]
new_hotels_BR['locale'] = 'BR'
new_hotels_BR['campaign'] = 1
new_hotels_BR = new_hotels_BR[['locale', 'partner_reference', 'campaign']].copy()
new_hotels_BR.reset_index(drop = True, inplace = True)
display(new_hotels_BR)

# US
new_hotels_US = new_hotels_US[['partner_reference']]
new_hotels_US['locale'] = 'US'
new_hotels_US['campaign'] = 1
new_hotels_US = new_hotels_US[['locale', 'partner_reference', 'campaign']].copy()
new_hotels_US.reset_index(drop = True, inplace = True)
display(new_hotels_US)

,locale,partner_reference,campaign


,locale,partner_reference,campaign


In [15]:
new_hotels_BR

,locale,partner_reference,campaign


In [16]:
new_hotels_US

,locale,partner_reference,campaign


In [17]:
# Concatenating US and BR datasets
new_hotels_output = pd.concat([new_hotels_BR, new_hotels_US])
new_hotels_output

,locale,partner_reference,campaign


In [18]:
### csv outputs (append these to the current feed_cpa)
new_hotels_output.to_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/novos hoteis/novos_hoteis.csv' )